In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create and go to a safe working folder inside your Drive
%cd /content/drive/MyDrive
!mkdir -p COEN691/SSD_Project
%cd COEN691/SSD_Project


Mounted at /content/drive
/content/drive/MyDrive
/content/drive/MyDrive/COEN691/SSD_Project


In [ ]:
!pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip -q install albumentations pycocotools opencv-python tqdm matplotlib pandas


In [ ]:
import torch, platform
print("CUDA available:", torch.cuda.is_available())
print("Torch:", torch.__version__, "| Python:", platform.python_version())


CUDA available: False
Torch: 2.8.0+cu126 | Python: 3.12.12


In [ ]:
!mkdir -p data/raw/Images data/raw/Annotation data/coco models/checkpoints src


In [ ]:
# Download the dataset archives (≈ 750 MB each)
!wget -q http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
!wget -q http://vision.stanford.edu/aditya86/ImageNetDogs/annotation.tar

# Unpack them into our data/raw folder
!tar -xf images.tar -C data/raw/
!tar -xf annotation.tar -C data/raw/

# Check the folders
!ls data/raw/Images | head
!ls data/raw/Annotation | head


n02085620-Chihuahua
n02085782-Japanese_spaniel
n02085936-Maltese_dog
n02086079-Pekinese
n02086240-Shih-Tzu
n02086646-Blenheim_spaniel
n02086910-papillon
n02087046-toy_terrier
n02087394-Rhodesian_ridgeback
n02088094-Afghan_hound
n02085620-Chihuahua
n02085782-Japanese_spaniel
n02085936-Maltese_dog
n02086079-Pekinese
n02086240-Shih-Tzu
n02086646-Blenheim_spaniel
n02086910-papillon
n02087046-toy_terrier
n02087394-Rhodesian_ridgeback
n02088094-Afghan_hound


In [ ]:
%%writefile src/prepare_dataset.py
import os, json, random, xml.etree.ElementTree as ET
from pathlib import Path

RAW_IMG_DIR = Path("data/raw/Images/")
RAW_ANN_DIR = Path("data/raw/Annotation/")
print("IMG DIR:", RAW_IMG_DIR.resolve())
print("ANN DIR:", RAW_ANN_DIR.resolve())

COCO_DIR    = Path("data/coco"); COCO_DIR.mkdir(parents=True, exist_ok=True)

def collect_pairs():
    pairs = []
    for breed in sorted(os.listdir(RAW_ANN_DIR)):
        ann_dir = RAW_ANN_DIR/breed
        img_dir = RAW_IMG_DIR/breed
        if not ann_dir.is_dir():
            continue
        for xml in ann_dir.glob("*.xml"):
            img = img_dir/(xml.stem + ".jpg")
            if img.exists():
                pairs.append((img, xml))
    return pairs

def parse_xml(xml_path):
    root = ET.parse(xml_path).getroot()
    bbox = root.find("object/bndbox")
    xmin = float(bbox.find("xmin").text); ymin = float(bbox.find("ymin").text)
    xmax = float(bbox.find("xmax").text); ymax = float(bbox.find("ymax").text)
    width = int(root.find("size/width").text); height = int(root.find("size/height").text)
    x, y, w, h = xmin, ymin, xmax - xmin, ymax - ymin
    return x, y, w, h, width, height

def main():
    pairs = collect_pairs()
    random.seed(42); random.shuffle(pairs)
    n = len(pairs)
    train = pairs[:int(0.7*n)]
    val   = pairs[int(0.7*n):int(0.85*n)]
    test  = pairs[int(0.85*n):]

    breeds = sorted([b for b in os.listdir(RAW_IMG_DIR) if (RAW_IMG_DIR/b).is_dir()])
    cat_id = {name: i+1 for i, name in enumerate(breeds)}
    categories = [{"id": i+1, "name": b, "supercategory": "dog"} for i, b in enumerate(breeds)]

    def make_split(name, split_pairs):
        images, annotations = [], []
        img_id = 1; ann_id = 1
        for img_path, xml_path in split_pairs:
            x, y, w, h, W, H = parse_xml(xml_path)
            images.append({"id": img_id, "file_name": str(img_path),
                           "width": W, "height": H})
            breed = xml_path.parent.name
            annotations.append({"id": ann_id, "image_id": img_id,
                                "category_id": cat_id[breed],
                                "bbox": [x,y,w,h], "area": w*h, "iscrowd": 0})
            img_id += 1; ann_id += 1
        coco = {"images": images, "annotations": annotations, "categories": categories}
        with open(COCO_DIR/f"{name}.json","w") as f: json.dump(coco,f)
        print(f"{name}: {len(images)} images, {len(annotations)} anns")

    make_split("train", train); make_split("val", val); make_split("test", test)

if __name__ == "__main__":
    main()


Writing src/prepare_dataset.py


In [ ]:
import os
print("Current folder:", os.getcwd())
!ls
!ls data
!ls data/raw


Current folder: /content/drive/MyDrive/COEN691/SSD_Project
annotation.tar	data  images.tar  models  src
coco  raw
Annotation  Images


In [ ]:
# Make sure the standard folders exist
!mkdir -p data/raw

# Move the top-level Images and Annotation into data/raw
# (the "|| true" part just avoids crashing if they already exist)
!mv Images data/raw/Images || true
!mv Annotation data/raw/Annotation || true

# Check the new structure
!ls data/raw
!ls data/raw/Images | head
!ls data/raw/Annotation | head


mv: cannot stat 'Images': No such file or directory
mv: cannot stat 'Annotation': No such file or directory
Annotation  Images
n02085620-Chihuahua
n02085782-Japanese_spaniel
n02085936-Maltese_dog
n02086079-Pekinese
n02086240-Shih-Tzu
n02086646-Blenheim_spaniel
n02086910-papillon
n02087046-toy_terrier
n02087394-Rhodesian_ridgeback
n02088094-Afghan_hound
n02085620-Chihuahua
n02085782-Japanese_spaniel
n02085936-Maltese_dog
n02086079-Pekinese
n02086240-Shih-Tzu
n02086646-Blenheim_spaniel
n02086910-papillon
n02087046-toy_terrier
n02087394-Rhodesian_ridgeback
n02088094-Afghan_hound


In [ ]:
!ls data/raw/Images | head
!ls data/raw/Annotation | head


n02085620-Chihuahua
n02085782-Japanese_spaniel
n02085936-Maltese_dog
n02086079-Pekinese
n02086240-Shih-Tzu
n02086646-Blenheim_spaniel
n02086910-papillon
n02087046-toy_terrier
n02087394-Rhodesian_ridgeback
n02088094-Afghan_hound
n02085620-Chihuahua
n02085782-Japanese_spaniel
n02085936-Maltese_dog
n02086079-Pekinese
n02086240-Shih-Tzu
n02086646-Blenheim_spaniel
n02086910-papillon
n02087046-toy_terrier
n02087394-Rhodesian_ridgeback
n02088094-Afghan_hound


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/COEN691/SSD_Project
import os
print("Current folder:", os.getcwd())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/COEN691/SSD_Project
Current folder: /content/drive/MyDrive/COEN691/SSD_Project


In [ ]:
!rm -rf data
!rm -f images.tar annotation.tar
!ls


models	src


In [ ]:
!mkdir -p data/raw
!ls data


raw


In [ ]:
!wget -q http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar -O images.tar
!wget -q http://vision.stanford.edu/aditya86/ImageNetDogs/annotation.tar -O annotation.tar

!ls -lh images.tar annotation.tar


-rw------- 1 root root  21M Oct 24  2011 annotation.tar
-rw------- 1 root root 757M Oct 20  2011 images.tar


In [ ]:
# This will create data/raw/Images and data/raw/Annotation
!tar -xf images.tar -C data/raw/
!tar -xf annotation.tar -C data/raw/

!ls data/raw
!ls data/raw/Images | head
!ls data/raw/Annotation | head


Annotation  Images
n02085620-Chihuahua
n02085782-Japanese_spaniel
n02085936-Maltese_dog
n02086079-Pekinese
n02086240-Shih-Tzu
n02086646-Blenheim_spaniel
n02086910-papillon
n02087046-toy_terrier
n02087394-Rhodesian_ridgeback
n02088094-Afghan_hound
n02085620-Chihuahua
n02085782-Japanese_spaniel
n02085936-Maltese_dog
n02086079-Pekinese
n02086240-Shih-Tzu
n02086646-Blenheim_spaniel
n02086910-papillon
n02087046-toy_terrier
n02087394-Rhodesian_ridgeback
n02088094-Afghan_hound


In [ ]:
!find data/raw/Annotation -maxdepth 3 -type f | head


data/raw/Annotation/n02085620-Chihuahua/n02085620_10074
data/raw/Annotation/n02085620-Chihuahua/n02085620_10131
data/raw/Annotation/n02085620-Chihuahua/n02085620_10621
data/raw/Annotation/n02085620-Chihuahua/n02085620_1073
data/raw/Annotation/n02085620-Chihuahua/n02085620_10976
data/raw/Annotation/n02085620-Chihuahua/n02085620_11140
data/raw/Annotation/n02085620-Chihuahua/n02085620_11238
data/raw/Annotation/n02085620-Chihuahua/n02085620_11258
data/raw/Annotation/n02085620-Chihuahua/n02085620_11337
data/raw/Annotation/n02085620-Chihuahua/n02085620_11477


In [ ]:
%%writefile src/prepare_dataset.py
import os, json, random, xml.etree.ElementTree as ET
from pathlib import Path

RAW_IMG_DIR = Path("data/raw/Images")
RAW_ANN_DIR = Path("data/raw/Annotation")
COCO_DIR    = Path("data/coco"); COCO_DIR.mkdir(parents=True, exist_ok=True)

def debug_check_dirs():
    print("IMG DIR:", RAW_IMG_DIR.resolve())
    print("ANN DIR:", RAW_ANN_DIR.resolve())
    print("IMG DIR exists:", RAW_IMG_DIR.exists())
    print("ANN DIR exists:", RAW_ANN_DIR.exists())
    if RAW_IMG_DIR.exists():
        breeds_img = [d.name for d in RAW_IMG_DIR.iterdir() if d.is_dir()]
        print("Sample image breeds:", breeds_img[:5])
    if RAW_ANN_DIR.exists():
        breeds_ann = [d.name for d in RAW_ANN_DIR.iterdir() if d.is_dir()]
        print("Sample annot breeds:", breeds_ann[:5])

def collect_pairs():
    pairs = []
    missing_img = 0
    total_xml = 0

    # 🔍 NEW: treat *any file* under Annotation as an XML (Stanford Dogs has no .xml extension)
    ann_files = [p for p in RAW_ANN_DIR.rglob("*") if p.is_file()]
    if not ann_files:
        print("[ERROR] No annotation files found anywhere under", RAW_ANN_DIR)
        return []

    print(f"[INFO] Total annotation files found (recursive): {len(ann_files)}")

    for ann_path in ann_files:
        total_xml += 1
        # Breed folder = top-level folder under Annotation (e.g., n02085620-Chihuahua)
        rel = ann_path.relative_to(RAW_ANN_DIR)
        breed_folder = rel.parts[0]

        img_dir = RAW_IMG_DIR / breed_folder

        stem = ann_path.name  # e.g., "n02085620_10074" (no extension)
        img = img_dir / (stem + ".jpg")
        if not img.exists():
            img_alt = img_dir / (stem + ".JPEG")
            if img_alt.exists():
                img = img_alt
            else:
                missing_img += 1
                # print(f"[MISS] No image for annotation: {ann_path}")
                continue

        pairs.append((img, ann_path))

    print(f"[SUMMARY] Total annotation files scanned: {total_xml}")
    print(f"[SUMMARY] Valid (img+ann) pairs: {len(pairs)}")
    print(f"[SUMMARY] Missing images for annotations: {missing_img}")
    return pairs

def parse_xml(xml_path):
    # Even without .xml extension, the content is XML, so ET.parse works fine
    root = ET.parse(xml_path).getroot()
    bbox = root.find("object/bndbox")
    xmin = float(bbox.find("xmin").text); ymin = float(bbox.find("ymin").text)
    xmax = float(bbox.find("xmax").text); ymax = float(bbox.find("ymax").text)
    width = int(root.find("size/width").text); height = int(root.find("size/height").text)
    x, y, w, h = xmin, ymin, xmax - xmin, ymax - ymin
    return x, y, w, h, width, height

def main():
    debug_check_dirs()

    pairs = collect_pairs()
    if not pairs:
        print("[ERROR] No (image, annotation) pairs found after recursive search.")
        return

    random.seed(42); random.shuffle(pairs)
    n = len(pairs)
    train = pairs[:int(0.7*n)]
    val   = pairs[int(0.7*n):int(0.85*n)]
    test  = pairs[int(0.85*n):]

    print(f"[SPLIT] Train: {len(train)}, Val: {len(val)}, Test: {len(test)}")

    # categories from image folders
    breeds = sorted([b for b in os.listdir(RAW_IMG_DIR) if (RAW_IMG_DIR/b).is_dir()])
    cat_id = {name: i+1 for i, name in enumerate(breeds)}
    categories = [{"id": i+1, "name": b, "supercategory": "dog"} for i, b in enumerate(breeds)]

    def make_split(name, split_pairs):
        images, annotations = [], []
        img_id = 1; ann_id = 1
        for img_path, ann_path in split_pairs:
            x, y, w, h, W, H = parse_xml(ann_path)
            images.append({
                "id": img_id,
                "file_name": str(img_path).replace("\\", "/"),
                "width": W,
                "height": H
            })
            breed_folder = ann_path.relative_to(RAW_ANN_DIR).parts[0]
            annotations.append({
                "id": ann_id,
                "image_id": img_id,
                "category_id": cat_id[breed_folder],
                "bbox": [x, y, w, h],
                "area": w*h,
                "iscrowd": 0
            })
            img_id += 1; ann_id += 1

        coco = {"images": images, "annotations": annotations, "categories": categories}
        out_path = COCO_DIR / f"{name}.json"
        with open(out_path, "w") as f:
            json.dump(coco, f)
        print(f"[WRITE] {name}: {len(images)} images, {len(annotations)} anns -> {out_path}")

    make_split("train", train)
    make_split("val", val)
    make_split("test", test)

if __name__ == "__main__":
    main()


Overwriting src/prepare_dataset.py


In [ ]:
!python src/prepare_dataset.py


IMG DIR: /content/drive/MyDrive/COEN691/SSD_Project/data/raw/Images
ANN DIR: /content/drive/MyDrive/COEN691/SSD_Project/data/raw/Annotation
IMG DIR exists: True
ANN DIR exists: True
Sample image breeds: ['n02085620-Chihuahua', 'n02085782-Japanese_spaniel', 'n02085936-Maltese_dog', 'n02086079-Pekinese', 'n02086240-Shih-Tzu']
Sample annot breeds: ['n02085620-Chihuahua', 'n02085782-Japanese_spaniel', 'n02085936-Maltese_dog', 'n02086079-Pekinese', 'n02086240-Shih-Tzu']
[INFO] Total annotation files found (recursive): 20580
[SUMMARY] Total annotation files scanned: 20580
[SUMMARY] Valid (img+ann) pairs: 20580
[SUMMARY] Missing images for annotations: 0
[SPLIT] Train: 14405, Val: 3088, Test: 3087
[WRITE] train: 14405 images, 14405 anns -> data/coco/train.json
[WRITE] val: 3088 images, 3088 anns -> data/coco/val.json
[WRITE] test: 3087 images, 3087 anns -> data/coco/test.json


In [ ]:
!ls -lh data/coco


total 4.7M
-rw------- 1 root root 719K Nov 13 18:19 test.json
-rw------- 1 root root 3.3M Nov 13 18:18 train.json
-rw------- 1 root root 720K Nov 13 18:18 val.json


In [ ]:
import json

# Load your COCO training annotation file
with open("data/coco/train.json", "r") as f:
    data = json.load(f)

# Create mapping from numeric id to breed name
id_to_name = {cat["id"]: cat["name"] for cat in data["categories"]}

# Show first few entries
list(id_to_name.items())[:10]


[(1, 'n02085620-Chihuahua'),
 (2, 'n02085782-Japanese_spaniel'),
 (3, 'n02085936-Maltese_dog'),
 (4, 'n02086079-Pekinese'),
 (5, 'n02086240-Shih-Tzu'),
 (6, 'n02086646-Blenheim_spaniel'),
 (7, 'n02086910-papillon'),
 (8, 'n02087046-toy_terrier'),
 (9, 'n02087394-Rhodesian_ridgeback'),
 (10, 'n02088094-Afghan_hound')]

In [ ]:
%cd /content/drive/MyDrive/COEN691/SSD_Project
!ls


/content/drive/MyDrive/COEN691/SSD_Project
annotation.tar	models		      ssd_detection_example.jpg
data		src		      ssd_labeled_bbox.jpg
images.tar	ssd_bbox_example.jpg  ssd_labeled_clean.jpg


In [ ]:
!zip -r src.zip src
!zip -r data_coco.zip data/coco


  adding: src/ (stored 0%)
  adding: src/prepare_dataset.py (deflated 63%)
  adding: src/__pycache__/ (stored 0%)
  adding: src/__pycache__/train_ssd.cpython-312.pyc (deflated 39%)
  adding: src/test_ssd.py (deflated 53%)
  adding: src/visualize_ssd.py (deflated 51%)
  adding: src/train_ssd.py (deflated 58%)
  adding: data/coco/ (stored 0%)
  adding: data/coco/train.json (deflated 85%)
  adding: data/coco/val.json (deflated 85%)
  adding: data/coco/test.json (deflated 85%)


In [ ]:
!zip -r checkpoints.zip models/checkpoints


  adding: models/checkpoints/ (stored 0%)
  adding: models/checkpoints/ssd_best.pth (deflated 8%)


In [ ]:
!zip ssd_detection_example.zip ssd_detection_example.jpg


  adding: ssd_detection_example.jpg (deflated 6%)


In [ ]:
!ls -lh *.zip


-rw------- 1 root root 140M Nov 16 08:49 checkpoints.zip
-rw------- 1 root root 704K Nov 16 08:48 data_coco.zip
-rw------- 1 root root  11K Nov 16 08:48 src.zip
-rw------- 1 root root 112K Nov 16 08:49 ssd_detection_example.zip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/COEN691/SSD_Project
!ls src


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/COEN691/SSD_Project
eval_ssd_anyhit.py  prepare_dataset.py	test_ssd.py   visualize_ssd.py
eval_ssd_top1.py    __pycache__		train_ssd.py


In [ ]:
import json
import math
from pathlib import Path

import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
from torchvision.transforms import functional as TF
from torchvision.models.detection import ssd300_vgg16
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 4          # keep small for stability
EPOCHS = 20             # you can change to 1 or 3
LR = 1e-4               # small LR to avoid exploding loss

class DogsCoco(CocoDetection):
    """
    Simple dataset:
    - uses CocoDetection to get (image, target)
    - converts COCO bbox [x,y,w,h] -> [x_min,y_min,x_max,y_max]
    - returns image tensor + target dict for SSD
    """

    def __init__(self, ann_file):
        super().__init__(root=".", annFile=ann_file, transforms=None)

    def __getitem__(self, idx):
        img, annotations_list = super().__getitem__(idx)
        img_np = img.convert("RGB")
        W, H = img_np.size # Image width and height from PIL Image

        boxes = []
        labels = []

        for ann in annotations_list:
            x_orig, y_orig, w_orig, h_orig = ann['bbox'] # COCO format: [x, y, width, height]

            # Convert to [x_min, y_min, x_max, y_max] format
            x_min = float(x_orig)
            y_min = float(y_orig)
            x_max = float(x_orig + w_orig)
            y_max = float(y_orig + h_orig)

            # Clamp coordinates to be within image boundaries [0, W] and [0, H]
            x_min = max(0.0, x_min)
            y_min = max(0.0, y_min)
            x_max = min(float(W), x_max)
            y_max = min(float(H), y_max)

            # Re-check for valid dimensions after clamping
            # If after clamping, width or height is non-positive, skip this box.
            if x_max <= x_min or y_max <= y_min:
                continue

            boxes.append([x_min, y_min, x_max, y_max])
            labels.append(ann['category_id'])

        # If no valid boxes remain after filtering, add a tiny dummy box
        if not boxes:
            x_min_dummy = W * 0.4
            y_min_dummy = H * 0.4
            x_max_dummy = W * 0.6
            y_max_dummy = H * 0.6
            # Ensure dummy box has positive width and height
            # This condition ensures that if W or H is 0, the dummy box is still valid if possible
            if x_max_dummy <= x_min_dummy: x_max_dummy = x_min_dummy + 1 # Add 1 to ensure positive width
            if y_max_dummy <= y_min_dummy: y_max_dummy = y_min_dummy + 1 # Add 1 to ensure positive height
            boxes.append([x_min_dummy, y_min_dummy, x_max_dummy, y_max_dummy])
            labels.append(1) # Assign to an arbitrary valid class ID (e.g., 1)

        # Convert image to tensor in [0,1], float32
        img_t = TF.to_tensor(img_np) # Converts to float32 in [0,1]
        boxes_t = torch.tensor(boxes, dtype=torch.float32)
        labels_t = torch.tensor(labels, dtype=torch.int64)

        target_dict = {
            "boxes": boxes_t,
            "labels": labels_t,
        }
        return img_t, target_dict

def collate_fn(batch):
    imgs, targets = list(zip(*batch))
    return list(imgs), list(targets)

def num_classes_from_json(path):
    with open(path, "r") as f:
        data = json.load(f)
    return len(data["categories"])

def main():
    print(f"Training on device: {DEVICE}")

    train_json = "data/coco/train.json"

    num_classes = num_classes_from_json(train_json)
    print("Number of dog breeds (classes):", num_classes)

    train_ds = DogsCoco(train_json)
    print("Train samples:", len(train_ds))

    train_loader = DataLoader(
        train_ds,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=2,
        pin_memory=False,
    )

    ckpt_dir = Path("models/checkpoints")
    ckpt_dir.mkdir(parents=True, exist_ok=True)
    ckpt_path = ckpt_dir / "ssd_best.pth"

    # Model
    model = ssd300_vgg16(weights=None, num_classes=num_classes + 1)
    model.to(DEVICE)

    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(
        params, lr=LR, momentum=0.9, weight_decay=5e-4
    )
    scheduler = StepLR(optimizer, step_size=10, gamma=0.1) # Moved scheduler.step() to after loss calculation, but before saving best model. Also set step_size = 10, instead of 1.

    best_loss = float("inf")

    for epoch in range(1, EPOCHS + 1):
        model.train()
        running_loss = 0.0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS} (train)")

        for images, targets in pbar:
            images = [img.to(DEVICE) for img in images]
            targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)
            loss = sum(loss_dict.values())
            loss_val = loss.item()

            # Skip bad batches with NaN/Inf loss
            if not math.isfinite(loss_val):
                print(f"⚠️  Skipping batch with non-finite loss: {loss_val}")
                continue

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss_val
            pbar.set_postfix(loss=f"{loss_val:.3f}")

        scheduler.step()

        avg_train_loss = running_loss / max(1, len(train_loader))
        print(f"Epoch {epoch}: avg train loss = {avg_train_loss:.4f}")

        # save model at end of each epoch if it's the best so far
        if avg_train_loss < best_loss:
            best_loss = avg_train_loss
            torch.save(model.state_dict(), ckpt_path)
            print(f"✅ Saved best model so far to {ckpt_path}")

if __name__ == "__main__":
    main()

Training on device: cuda
Number of dog breeds (classes): 120
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
Train samples: 14405


Epoch 1/20 (train): 100%|██████████| 3602/3602 [36:10<00:00,  1.66it/s, loss=9.702]


Epoch 1: avg train loss = 15.6374
✅ Saved best model so far to models/checkpoints/ssd_best.pth


Epoch 2/20 (train): 100%|██████████| 3602/3602 [10:36<00:00,  5.66it/s, loss=10.071]


Epoch 2: avg train loss = 8.2723
✅ Saved best model so far to models/checkpoints/ssd_best.pth


...

Epoch 18: avg train loss = 8.0572
✅ Saved best model so far to models/checkpoints/ssd_best.pth


Epoch 19/20 (train): 100%|██████████| 3602/3602 [10:36<00:00,  5.66it/s, loss=6.189]


Epoch 19: avg train loss = 8.0579


Epoch 20/20 (train): 100%|██████████| 3602/3602 [10:33<00:00,  5.68it/s, loss=10.197]


Epoch 20: avg train loss = 8.0559
✅ Saved best model so far to models/checkpoints/ssd_best.pth


In [ ]:
%%writefile src/eval_ssd_anyhit.py
import json
from pathlib import Path

import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
from torchvision.models.detection import ssd300_vgg16
from torchvision.transforms import functional as TF

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


class DogsCocoVal(CocoDetection):
    """
    COCO dataset wrapper for Stanford Dogs validation set.
    We only use the first annotation's category_id as the ground-truth breed label.
    """
    def __init__(self, img_root, ann_file):
        super().__init__(root=img_root, annFile=ann_file, transforms=None)

    def __getitem__(self, idx):
        img, target = super().__getitem__(idx)
        img = img.convert("RGB")
        img_t = TF.to_tensor(img)  # [3,H,W], float32 in [0,1]

        # target is a list of annotations for that image (dicts)
        if len(target) == 0:
            gt_label = -1  # no annotation
        else:
            # use first dog's category_id as GT breed label
            gt_label = int(target[0]["category_id"])

        return img_t, gt_label


def collate_fn(batch):
    imgs, labels = zip(*batch)
    return list(imgs), list(labels)


def main():
    ann_path = "data/coco/val.json"
    img_root = "."

    # read categories to know how many classes we have
    with open(ann_path, "r") as f:
        data = json.load(f)
    categories = data["categories"]
    num_classes = len(categories)
    print(f"Number of breeds (classes): {num_classes}")

    # dataset & loader
    ds = DogsCocoVal(img_root=img_root, ann_file=ann_path)
    loader = DataLoader(
        ds,
        batch_size=4,
        shuffle=False,
        collate_fn=collate_fn,
        num_workers=2,
    )
    print(f"Validation samples: {len(ds)}")

    # load model
    model = ssd300_vgg16(weights=None, num_classes=num_classes + 1)
    ckpt_path = "models/checkpoints/ssd_best.pth"
    print(f"Loading checkpoint from: {ckpt_path}")
    state_dict = torch.load(ckpt_path, map_location=DEVICE)
    model.load_state_dict(state_dict, strict=False)
    model.to(DEVICE)
    model.eval()

    total = 0
    correct = 0

    with torch.no_grad():
        for imgs, gt_labels in loader:
            imgs = [img.to(DEVICE) for img in imgs]
            outputs = model(imgs)

            for out, gt in zip(outputs, gt_labels):
                gt = int(gt)
                if gt == -1:
                    continue  # skip if no GT

                labels = out["labels"].cpu()

                # relaxed metric: if *any* predicted label equals GT, call it correct
                if (labels == gt).any():
                    correct += 1
                total += 1

    acc = correct / total if total > 0 else 0.0
    print("\nRelaxed metric: does ANY predicted label match the GT breed?")
    print(f"Accuracy_any_hit = {acc:.4f} ({correct}/{total})")


if __name__ == "__main__":
    main()


Writing src/eval_ssd_anyhit.py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/COEN691/SSD_Project

!python src/eval_ssd_anyhit.py


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/COEN691/SSD_Project
Number of breeds (classes): 120
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Validation samples: 3088
Loading checkpoint from: models/checkpoints/ssd_best.pth

Relaxed metric: does ANY predicted label match the GT breed?
Accuracy_any_hit = 0.0081 (25/3088)


In [ ]:
!find /content/drive/MyDrive/COEN691/SSD_Project -name "ssd_best.pth"


/content/drive/MyDrive/COEN691/SSD_Project/models/checkpoints/ssd_best.pth


In [ ]:
%%writefile src/test_ssd.py
import json
from pathlib import Path

import torch
from torchvision.transforms import functional as TF
from torchvision.models.detection import ssd300_vgg16
from PIL import Image

from train_ssd import num_classes_from_json  # reuse helper

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def main():
    # 1. Load one image path from train.json
    train_json = "data/coco/train.json"
    with open(train_json, "r") as f:
        data = json.load(f)

    first_img = data["images"][0]
    img_path = first_img["file_name"]
    print("Using image:", img_path)

    # 2. Load image
    img = Image.open(img_path).convert("RGB")
    img_t = TF.to_tensor(img).to(DEVICE).unsqueeze(0)  # [1,3,H,W]

    # 3. Build model and load weights
    num_classes = num_classes_from_json(train_json)
    model = ssd300_vgg16(weights=None, num_classes=num_classes + 1)
    ckpt_path = Path("models/checkpoints/ssd_best.pth")
    state_dict = torch.load(ckpt_path, map_location=DEVICE)
    model.load_state_dict(state_dict, strict=False)
    model.to(DEVICE)
    model.eval()

    # 4. Run prediction
    with torch.no_grad():
        outputs = model(img_t)

    preds = outputs[0]
    boxes = preds["boxes"]
    labels = preds["labels"]
    scores = preds["scores"]

    # 5. Print top 5 predictions
    print("Top 5 detections:")
    for i in range(min(5, len(boxes))):
        breed_name = id_to_name.get(labels[i].item(), "Unknown")
print(f"- box={boxes[i].cpu().numpy()}, breed={breed_name}, score={scores[i].item():.3f}")

if __name__ == "__main__":
    main()


Writing src/test_ssd.py


In [ ]:
!python src/test_ssd.py


Using image: data/raw/Images/n02106382-Bouvier_des_Flandres/n02106382_4243.jpg
Top 5 detections:
- box=[ 60.16299   34.112934 518.9856   322.909   ], label_id=5, score=0.012
- box=[ 60.16299   34.112934 518.9856   322.909   ], label_id=109, score=0.011
- box=[ 63.47534   33.976807 512.04926  320.42667 ], label_id=89, score=0.011
- box=[ 60.16299   34.112934 518.9856   322.909   ], label_id=27, score=0.011
- box=[ 60.16299   34.112934 518.9856   322.909   ], label_id=3, score=0.011


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/COEN691/SSD_Project
!pwd
!ls


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/COEN691/SSD_Project
/content/drive/MyDrive/COEN691/SSD_Project
annotation.tar	 images.tar  ssd_bbox_example.jpg	ssd_labeled_clean.jpg
checkpoints.zip  models      ssd_detection_example.jpg
data		 src	     ssd_detection_example.zip
data_coco.zip	 src.zip     ssd_labeled_bbox.jpg


In [ ]:
!zip -r ssd_artifacts.zip \
  src \
  models/checkpoints/ssd_best.pth \
  data/coco/train.json \
  data/coco/val.json \
  data/coco/test.json


updating: src/ (stored 0%)
updating: src/prepare_dataset.py (deflated 63%)
updating: src/__pycache__/ (stored 0%)
updating: src/__pycache__/train_ssd.cpython-312.pyc (deflated 39%)
updating: src/test_ssd.py (deflated 53%)
updating: src/visualize_ssd.py (deflated 51%)
updating: src/train_ssd.py (deflated 58%)
updating: src/eval_ssd_top1.py (deflated 59%)
updating: src/eval_ssd_anyhit.py (deflated 57%)
updating: models/checkpoints/ssd_best.pth (deflated 8%)
updating: data/coco/train.json (deflated 85%)
updating: data/coco/val.json (deflated 85%)
updating: data/coco/test.json (deflated 85%)
